In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.impute import SimpleImputer
import os
import pandas as pd

In [2]:
# Set working directory
os.chdir('C:\\Users\\Chand\\Downloads\\Assignment1')
print(os.getcwd())

C:\Users\Chand\Downloads\Assignment1


In [3]:
# Load the dataset
data = pd.read_csv("NSSO68.csv")

C:\Users\Chand\AppData\Local\Temp\ipykernel_15052\95208774.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


In [26]:
# Subset data to state assigned
subset_data = data[data['state_1'] == 'MIZ'][['foodtotal_v', 'hhdsz', 'Regular_salary_earner', 'MPCE_MRP', 'MPCE_URP', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)

       foodtotal_v  hhdsz  Regular_salary_earner  MPCE_MRP  MPCE_URP  \
14581   968.718500      4                    2.0   2925.13   3449.75   
14582  1039.043333      3                    2.0   2854.06   3621.00   
14583   766.020714      7                    2.0   2055.04   2026.00   
14584   744.270000      2                    1.0   2658.94   2562.50   
14585   900.351667      3                    2.0   1993.71   1943.67   
...            ...    ...                    ...       ...       ...   
47552   450.515833      6                    2.0    845.74    824.50   
47553   542.277000      5                    2.0   1011.32    882.20   
47554   448.071250      4                    2.0   1008.17   1023.00   
47555   468.479000      5                    2.0    943.10    847.40   
47556   538.958750      4                    2.0   1062.15    981.50   

       Possess_ration_card  Education  No_of_Meals_per_day  
14581                  1.0        8.0                  NaN  
14582        

In [22]:
# Check for missing values
print(subset_data['hhdsz'].isna().sum())
print(subset_data['Regular_salary_earner'].isna().sum())
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(subset_data['Education'].isna().sum())
print(subset_data['No_of_Meals_per_day'].isna().sum())

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
subset_data['Possess_ration_card'] = imputer.fit_transform(subset_data[['Possess_ration_card']])

print("Possess_ration_card:")
print(subset_data['Possess_ration_card'].isna().sum())

0
0
0
0
2
0
0
Possess_ration_card:
0


In [23]:
# Fit the regression model
X = subset_data[['hhdsz', 'Regular_salary_earner', 'MPCE_MRP', 'MPCE_URP', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = subset_data['foodtotal_v']

model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_v   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     1302.
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        21:16:31   Log-Likelihood:                -61381.
No. Observations:                9015   AIC:                         1.228e+05
Df Residuals:                    9007   BIC:                         1.228e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   361.01

In [24]:
# multicollinearity using Variance Inflation Factor (VIF)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)  # VIF Value more than 8 is problematic

                 feature         VIF
0                  const  105.477846
1                  hhdsz    1.098855
2  Regular_salary_earner    1.138218
3               MPCE_MRP    2.068354
4               MPCE_URP    1.968635
5    Possess_ration_card    1.048881
6              Education    1.230296
7    No_of_Meals_per_day    1.004672


In [25]:
# Extract the coefficients from the model
coefficients = model.params

# Construct the equation
equation = f"y = {coefficients[0]:.2f}"
for i in range(1, len(coefficients)):
    equation += f" + {coefficients[i]:.6f}*x{i}"

# Print the equation
print(equation)

y = 361.02 + -12.963010*x1 + -14.608830*x2 + 0.072781*x3 + 0.059190*x4 + -48.084503*x5 + 7.634267*x6 + 49.872590*x7
